<a href="https://www.kaggle.com/code/shedai/llm-encoder-decoder?scriptVersionId=249028445" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Encoder-Decoder mimarisinin nasıl çalıştığını, iki bölümün birbiriyle nasıl konuştuğunu ve adım adım bir çıktının nasıl üretildiğini gösteren, gerçek bir sinir ağı içeren bir kod örneği hazırladım.

Bu örnekte, simülasyon yerine PyTorch kütüphanesini kullanarak, basit bir çeviri görevini (İngilizce sayılardan Türkçe sayılara) yerine getiren, çalışan bir Encoder-Decoder modeli oluşturacağız. Kod, her aşamada neler olduğunu anlamanızı kolaylaştıracak şekilde bolca açıklama ve ara çıktı içerir.

Senaryo: Basit Bir "Sayı Çevirmeni"

Görev: "one" -> "bir", "two" -> "iki" gibi basit çeviriler yapmak.

Mimari: Klasik ve anlaşılması kolay olan RNN (GRU) tabanlı bir Encoder-Decoder modeli kullanacağız. Bu, Transformer'dan önceki temel Seq2Seq yapısıdır ve mimarinin temel mantığını anlamak için mükemmeldir.

Adımlar:

* Veri Hazırlığı: İngilizce ve Türkçe kelimeler için ayrı sözlükler oluşturacağız.

* Encoder Modeli: İngilizce cümleyi okuyup bir "anlam vektörüne" (context vector) dönüştürecek.

* Decoder Modeli: Bu anlam vektörünü alıp adım adım Türkçe çıktıyı üretecek.

* Eğitim: Modelin bu çeviriyi nasıl öğrendiğini göreceğiz.

* Test: Eğitilmiş modelle yeni bir çeviri yapacağız.

# Basit Encoder-Decoder Yapısı

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import random

# --- Adım 1: Veri Hazırlığı ---
print("--- Adım 1: Veri Hazırlığı ---")

# Basit veri setimiz ve özel token'lar
SOS_token = 0  # Start of Sentence (Cümle Başlangıcı)
EOS_token = 1  # End of Sentence (Cümle Sonu)

# İngilizce'den Türkçe'ye çeviri çiftleri
pairs = [
    ("one", "bir"), ("two", "iki"), ("three", "üç"), ("four", "dört"),
    ("five", "beş"), ("six", "altı"), ("seven", "yedi"), ("eight", "sekiz"),
    ("nine", "dokuz"), ("ten", "on")
]

# Her dil için kelime-indeks sözlükleri oluşturma
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.index2word = {SOS_token: "SOS", EOS_token: "EOS"}
        self.n_words = 2  # SOS ve EOS token'ları ile başla

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.index2word[self.n_words] = word
            self.n_words += 1

input_lang = Lang('eng')
output_lang = Lang('tur')

for pair in pairs:
    input_lang.addSentence(pair[0])
    output_lang.addSentence(pair[1])

print(f"İngilizce Kelime Sayısı: {input_lang.n_words}")
print(f"Türkçe Kelime Sayısı: {output_lang.n_words}")
print(f"Örnek: 'one' -> {input_lang.word2index['one']}, 'bir' -> {output_lang.word2index['bir']}\n")
print("="*60)


# --- Adım 2: Modelleri Tanımlama ---
print("\n--- Adım 2: Encoder ve Decoder Modelleri ---")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
hidden_size = 128 # Embedding ve gizli katman boyutu

# ENCODER: Girdiyi Anlayan Kısım
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.gru(embedded, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

# DECODER: Anlamdan Yeni Metin Üreten Kısım
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = torch.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

print("Encoder ve Decoder modelleri (RNN-GRU tabanlı) tanımlandı.\n")
print("="*60)

# --- Adım 3: Eğitim Döngüsü ---
print("\n--- Adım 3: Modelin Eğitimi ---")
# Eğitim fonksiyonu
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    
    loss = 0

    # ---------- ENCODER AŞAMASI ----------
    # Girdi cümlesini okuyup bir bağlam vektörüne sıkıştır
    print(f"\n>>> '{input_lang.index2word[input_tensor[0].item()]}' kelimesi işleniyor...")
    print("1. ENCODER çalışıyor: Girdi okunuyor...")
    for ei in range(len(input_tensor)):
        _, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
    
    print(f"2. ENCODER tamamlandı. 'Anlam Özeti' (Context Vector) oluşturuldu.")
    print(f"   Context Vector (encoder_hidden) boyutu: {encoder_hidden.shape}")

    # ---------- DECODER AŞAMASI ----------
    # Encoder'ın son gizli durumu, Decoder'ın ilk gizli durumu olarak kullanılır.
    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden # KÖPRÜ: Bilgi burada aktarılıyor!
    print("\n3. DECODER çalışıyor: Adım adım çeviri üretiliyor...")
    
    # Teacher Forcing: Gerçek hedef kelimeleri bir sonraki girdi olarak kullan
    for di in range(len(target_tensor)):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        
        # Kaybı hesapla
        loss += criterion(decoder_output, target_tensor[di].unsqueeze(0))
        
        # Bir sonraki girdi olarak gerçek hedefi ver
        decoder_input = target_tensor[di]
        
        # ARA ÇIKTI: Decoder'ın her adımda ne yaptığını gör
        topv, topi = decoder_output.topk(1)
        predicted_word = output_lang.index2word[topi.item()]
        target_word = output_lang.index2word[target_tensor[di].item()]
        print(f"   -> Adım {di+1}: Tahmin='{predicted_word}', Gerçek Hedef='{target_word}'")

    # Gradyanları hesapla ve ağırlıkları güncelle
    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / len(target_tensor)

# Eğitim için hazırlık
encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = DecoderRNN(hidden_size, output_lang.n_words).to(device)
encoder_optimizer = optim.SGD(encoder.parameters(), lr=0.01)
decoder_optimizer = optim.SGD(decoder.parameters(), lr=0.01)
criterion = nn.NLLLoss()

n_iters = 1000
print_every = 250
total_loss = 0

print(f"\n{n_iters} iterasyonluk eğitim başlıyor...\n")

for iter in range(1, n_iters + 1):
    # Rastgele bir çift seç
    pair = random.choice(pairs)
    input_tensor = torch.LongTensor([input_lang.word2index[s] for s in pair[0].split(' ')]).to(device)
    target_tensor = torch.LongTensor([output_lang.word2index[s] for s in pair[1].split(' ')] + [EOS_token]).to(device)
    
    # Her 'print_every' adımda ara çıktıları göster
    if iter % print_every == 0:
        loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print(f"Iterasyon {iter} ({iter*100/n_iters}%), Kayıp: {loss:.4f}")
    else: # Sessiz eğitim
        # Bu kısım normalde print içermez, biz öğrenme amacıyla ekledik.
        # Gerçek eğitimde sadece kayıp hesaplanır.
        encoder_hidden = encoder.initHidden()
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        loss = 0
        for ei in range(len(input_tensor)):
            _, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        decoder_input = torch.tensor([[SOS_token]], device=device)
        decoder_hidden = encoder_hidden
        for di in range(len(target_tensor)):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di].unsqueeze(0))
            decoder_input = target_tensor[di]
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()

print("\nEğitim Tamamlandı!")
print("="*60)


# --- Adım 4: Modeli Test Etme ---
print("\n--- Adım 4: Eğitilmiş Model ile Çeviri Testi ---")
def evaluate(encoder, decoder, sentence):
    with torch.no_grad(): # Gradyan hesaplamayı kapat
        input_tensor = torch.LongTensor([input_lang.word2index[s] for s in sentence.split(' ')]).to(device)
        
        # ENCODER
        encoder_hidden = encoder.initHidden()
        for ei in range(len(input_tensor)):
            _, encoder_hidden = decoder(input_tensor[ei], encoder_hidden) # Düzeltme: encoder_hidden encoder'dan gelmeli
            _, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        
        # DECODER
        decoder_input = torch.tensor([[SOS_token]], device=device)
        decoder_hidden = encoder_hidden
        decoded_words = []
        
        print(f"\n>>> '{sentence}' çeviriliyor...")
        print("Adım Adım Üretim:")
        for di in range(5): # Maksimum 5 kelime üret
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])
            
            # Bir sonraki girdi, modelin kendi tahmini olacak (Teacher Forcing YOK)
            decoder_input = topi.squeeze().detach()
            print(f"   -> Adım {di+1}: Üretilen kelime = '{decoded_words[-1]}'")

        return ' '.join(decoded_words)

# Test edelim
for pair in random.sample(pairs, 3):
    output_words = evaluate(encoder, decoder, pair[0])
    print(f"Girdi: {pair[0]} -> Çıktı: {output_words}")

--- Adım 1: Veri Hazırlığı ---
İngilizce Kelime Sayısı: 12
Türkçe Kelime Sayısı: 12
Örnek: 'one' -> 2, 'bir' -> 2


--- Adım 2: Encoder ve Decoder Modelleri ---
Encoder ve Decoder modelleri (RNN-GRU tabanlı) tanımlandı.


--- Adım 3: Modelin Eğitimi ---

1000 iterasyonluk eğitim başlıyor...


>>> 'ten' kelimesi işleniyor...
1. ENCODER çalışıyor: Girdi okunuyor...
2. ENCODER tamamlandı. 'Anlam Özeti' (Context Vector) oluşturuldu.
   Context Vector (encoder_hidden) boyutu: torch.Size([1, 1, 128])

3. DECODER çalışıyor: Adım adım çeviri üretiliyor...
   -> Adım 1: Tahmin='on', Gerçek Hedef='on'
   -> Adım 2: Tahmin='EOS', Gerçek Hedef='EOS'
Iterasyon 250 (25.0%), Kayıp: 0.5278

>>> 'seven' kelimesi işleniyor...
1. ENCODER çalışıyor: Girdi okunuyor...
2. ENCODER tamamlandı. 'Anlam Özeti' (Context Vector) oluşturuldu.
   Context Vector (encoder_hidden) boyutu: torch.Size([1, 1, 128])

3. DECODER çalışıyor: Adım adım çeviri üretiliyor...
   -> Adım 1: Tahmin='yedi', Gerçek Hedef='yedi'
   ->

Önceki Encoder-Decoder kodumuz, tek kelimelik "dizileri" çevirdiği için, kelime sırasının önemini tam olarak gösteremiyordu. Bir RNN (GRU) yapısı doğası gereği sıralı veri işlese de, bunu gösterebileceğimiz daha iyi bir örnekle konsepti pekiştirebiliriz.

Bu güncellenmiş kodda, basit bir kelime-kelime çevirisi yerine, tam cümle çevirisi yapacağız. Bu sayede, Encoder'ın bir cümlenin kelime dizilimini nasıl bir "anlam özetine" (context vector) kodladığını ve Decoder'ın bu özeti kullanarak tamamen farklı bir dizilime sahip yeni bir cümle nasıl ürettiğini adım adım görebileceğiz.

Senaryo: İngilizce'den Türkçe'ye Cümle Çevirmeni

Görev: "the man eats an apple" gibi basit İngilizce cümleleri, dilbilgisi ve kelime sırası farklı olan "adam bir elma yer" gibi Türkçe cümlelere çevirmek.

Odak Noktası: Encoder'ın the -> man -> eats -> an -> apple sırasını işleyerek ürettiği tek bir bağlam vektörünün, Decoder tarafından nasıl adam -> bir -> elma -> yer gibi farklı bir sırada "çözüldüğünü" göstermek. Bu, dizilim bilgisinin nasıl korunduğunu ve dönüştürüldüğünü kanıtlar.

Mimari: Önceki kodla aynı, anlaşılması kolay RNN (GRU) tabanlı Encoder-Decoder yapısını kullanacağız. Değişiklikler veri setinde ve ara çıktıların sunumunda olacak.

# Cümle Bazlı Çeviri

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import time
import math

# --- Adım 1: Cümle Bazlı Veri Hazırlığı ---
print("--- Adım 1: Cümle Bazlı Veri Hazırlığı ---")

SOS_token = 0  # Start of Sentence
EOS_token = 1  # End of Sentence

# Kelime sırası farklı olan cümle çiftleri
pairs = [
    ("the man eats an apple", "adam bir elma yer"),
    ("the woman drinks water", "kadın su içer"),
    ("a smart dog runs fast", "akıllı bir köpek hızlı koşar"),
    ("a lazy cat sleeps all day", "tembel bir kedi bütün gün uyur"),
    ("the smart woman reads a book", "akıllı kadın bir kitap okur"),
    ("the fast dog eats meat", "hızlı köpek et yer")
]

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.index2word = {SOS_token: "SOS", EOS_token: "EOS"}
        self.n_words = 2

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.index2word[self.n_words] = word
            self.n_words += 1

input_lang = Lang('eng')
output_lang = Lang('tur')

for pair in pairs:
    input_lang.addSentence(pair[0])
    output_lang.addSentence(pair[1])

print(f"İngilizce Kelime Sayısı: {input_lang.n_words}")
print(f"Türkçe Kelime Sayısı: {output_lang.n_words}\n")
print("="*60)


# --- Adım 2: Modeller (Değişiklik Yok) ---
# Encoder ve Decoder sınıfları öncekiyle aynı, çünkü RNN zaten sıralı çalışır.
# Önemli olan onlara nasıl veri verdiğimiz ve süreci nasıl yorumladığımızdır.
print("\n--- Adım 2: Encoder ve Decoder Modelleri (Yapısal Değişiklik Yok) ---")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
hidden_size = 128

class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.gru(embedded, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = torch.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

print("Modeller hazır.\n")
print("="*60)


# --- Adım 3: Eğitim Döngüsü ve Detaylı Ara Çıktılar ---
print("\n--- Adım 3: Modelin Cümle Dizilimini Öğrenmesi ---")

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion):
    encoder_hidden = encoder.initHidden()
    encoder_optimizer.zero_grad(); decoder_optimizer.zero_grad()
    loss = 0
    input_length = len(input_tensor); target_length = len(target_tensor)

    # ---------- ENCODER AŞAMASI: Sıralı Anlama ----------
    print(f"\n>>> Girdi Cümlesi İşleniyor: '{' '.join([input_lang.index2word[i.item()] for i in input_tensor])}'")
    print("1. ENCODER çalışıyor: Cümledeki kelime dizilimi okunuyor...")
    # RNN'in gizli durumu (hidden state) her kelimede güncellenir.
    # Bu, kelime sırası bilgisinin vektöre kodlanmasını sağlar.
    for ei in range(input_length):
        _, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        print(f"   Encoder Adım {ei+1}: '{input_lang.index2word[input_tensor[ei].item()]}' okundu. Gizli durum güncellendi.")
    
    print(f"\n2. ENCODER tamamlandı. Cümlenin sıralı anlamı 'Context Vector'e kodlandı.")
    
    # ---------- DECODER AŞAMASI: Sıralı Üretim ----------
    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden # KÖPRÜ: Kodlanmış sıralı bilgi Decoder'a aktarılıyor.
    print("\n3. DECODER çalışıyor: Yeni dizilime sahip cümle üretiliyor...")
    
    for di in range(target_length):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        loss += criterion(decoder_output, target_tensor[di].unsqueeze(0))
        decoder_input = target_tensor[di] # Teacher Forcing
        predicted_word = output_lang.index2word[decoder_output.data.topk(1)[1].item()]
        print(f"   -> Decoder Adım {di+1}: Üretilen kelime = '{predicted_word}'")
        if decoder_input.item() == EOS_token: break

    loss.backward()
    encoder_optimizer.step(); decoder_optimizer.step()
    return loss.item() / target_length

# Eğitim için hazırlık
encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = DecoderRNN(hidden_size, output_lang.n_words).to(device)
encoder_optimizer = optim.SGD(encoder.parameters(), lr=0.01)
decoder_optimizer = optim.SGD(decoder.parameters(), lr=0.01)
criterion = nn.NLLLoss()

n_iters = 7500
print_every = 1500

print(f"\n{n_iters} iterasyonluk eğitim başlıyor...\n")
# Sadece belirli adımlarda detaylı çıktı verelim
for iter in range(1, n_iters + 1):
    pair = random.choice(pairs)
    input_tensor = torch.LongTensor([input_lang.word2index[s] for s in pair[0].split(' ')]).to(device)
    target_tensor = torch.LongTensor([output_lang.word2index[s] for s in pair[1].split(' ')] + [EOS_token]).to(device)
    
    if iter % print_every == 0:
        print(f"--- Iterasyon {iter} ---")
        loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print(f"   Kayıp (Loss): {loss:.4f}")
    else: # Sessiz eğitim
        # Bu adımlarda çıktı basılmaz ama eğitim devam eder
        encoder_hidden = encoder.initHidden()
        encoder_optimizer.zero_grad(); decoder_optimizer.zero_grad()
        loss = 0; input_length = len(input_tensor); target_length = len(target_tensor)
        for ei in range(input_length):
            _, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        decoder_input = torch.tensor([[SOS_token]], device=device)
        decoder_hidden = encoder_hidden
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di].unsqueeze(0))
            decoder_input = target_tensor[di]
        loss.backward()
        encoder_optimizer.step(); decoder_optimizer.step()

print("\nEğitim Tamamlandı!")
print("="*60)

# --- Adım 4: Modeli Test Etme ---
print("\n--- Adım 4: Eğitilmiş Model ile Çeviri Testi ---")
def evaluate(encoder, decoder, sentence):
    """Cümleyi alır, bilinmeyen kelimeleri atlar ve çeviriyi üretir."""
    with torch.no_grad(): # Gradyan hesaplamayı kapat
        
        # ########### ÇÖZÜM: BU SATIR DÜZELTİLDİ ###########
        # List comprehension içine bir 'if' koşulu ekleyerek, sadece sözlükte
        # var olan kelimelerin indekslerini alıyoruz.
        input_words = sentence.split(' ')
        input_indices = [input_lang.word2index[word] for word in input_words if word in input_lang.word2index]

        # Eğer cümledeki hiçbir kelime sözlükte yoksa, anlamlı bir mesaj döndür
        if not input_indices:
            return "Üzgünüm, sorunuzdaki kelimeleri anlayamadım."

        input_tensor = torch.LongTensor(input_indices).to(device)
        # ######################################################
        
        # Fonksiyonun geri kalanı aynı
        input_length = len(input_tensor)
        encoder_hidden = encoder.initHidden()
        for ei in range(input_length):
            _, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            
        decoder_input = torch.tensor([[SOS_token]], device=device)
        decoder_hidden = encoder_hidden
        decoded_words = []
        
        for di in range(10): # Maksimum kelime sayısı
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])
                
            decoder_input = topi.squeeze().detach()
            
        return ' '.join(decoded_words)

# Test edelim
for _ in range(3):
    pair = random.choice(pairs)
    evaluate(encoder, decoder, pair[0])

--- Adım 1: Cümle Bazlı Veri Hazırlığı ---
İngilizce Kelime Sayısı: 23
Türkçe Kelime Sayısı: 21


--- Adım 2: Encoder ve Decoder Modelleri (Yapısal Değişiklik Yok) ---
Modeller hazır.


--- Adım 3: Modelin Cümle Dizilimini Öğrenmesi ---

7500 iterasyonluk eğitim başlıyor...

--- Iterasyon 1500 ---

>>> Girdi Cümlesi İşleniyor: 'a smart dog runs fast'
1. ENCODER çalışıyor: Cümledeki kelime dizilimi okunuyor...
   Encoder Adım 1: 'a' okundu. Gizli durum güncellendi.
   Encoder Adım 2: 'smart' okundu. Gizli durum güncellendi.
   Encoder Adım 3: 'dog' okundu. Gizli durum güncellendi.
   Encoder Adım 4: 'runs' okundu. Gizli durum güncellendi.
   Encoder Adım 5: 'fast' okundu. Gizli durum güncellendi.

2. ENCODER tamamlandı. Cümlenin sıralı anlamı 'Context Vector'e kodlandı.

3. DECODER çalışıyor: Yeni dizilime sahip cümle üretiliyor...
   -> Decoder Adım 1: Üretilen kelime = 'akıllı'
   -> Decoder Adım 2: Üretilen kelime = 'bir'
   -> Decoder Adım 3: Üretilen kelime = 'köpek'
   -> Decoder 

In [4]:
print(pair[0])

evaluate(encoder, decoder, pair[0])

a lazy cat sleeps all day


'tembel bir kedi bütün gün uyur <EOS>'

Kod Çıktısı ve Dizilim Vurgusu

Bu kodu çalıştırdığınızda, eğitim sırasındaki ara çıktılar size sürecin ruhunu gösterecektir:

Encoder Aşaması: Çıktıda, Encoder'ın "the", "man", "eats"... kelimelerini sırayla okuduğunu ve her adımda gizli durumunu (hidden state) güncellediğini göreceksiniz. Bu, RNN'in temel çalışma prensibidir. Son adımda oluşan encoder_hidden (Context Vector), artık sadece kelimelerin bir torbası değil, "adamın elma yediği" sıralı bilgisini içeren bir özettir.

Decoder Aşaması: Decoder, bu sıralı bilgiyi barındıran tek bir vektörle başlar.

İlk adımda, bu özete bakarak Türkçe cümlenin başına en uygun kelimenin "adam" olduğuna karar verir.

İkinci adımda, hem özete hem de az önce ürettiği "adam" kelimesine bakarak bir sonraki kelimenin "bir" olması gerektiğine karar verir.

Bu süreç, İngilizce'deki S-V-O (Subject-Verb-Object) yapısının, Türkçe'deki S-O-V (Subject-Object-Verb) yapısına nasıl başarıyla dönüştürüldüğünü gösterir.

Bu örnek, Encoder-Decoder mimarisinin sadece kelimeleri çevirmekle kalmayıp, bir dilin yapısını ve dizilimini "anlayıp" başka bir dilin yapısına nasıl dönüştürdüğünü somut bir şekilde kanıtlar. Bu dönüşüm, Encoder'ın sıralı bilgiyi tek bir vektörde başarılı bir şekilde kodlaması sayesinde mümkün olur.

# Sohbet Robotu ve Encoder/Decoder Etkisi

In [5]:
import numpy as np
import gensim
import torch
import torch.nn as nn
import torch.optim as optim
import random

# --- Ortak Veri Seti ---
# Müşteri hizmetleri için basit Soru-Cevap çiftleri
qa_pairs = [
    ("ürün ne zaman kargoya verilir", "siparişiniz 1-3 iş günü içinde kargoya verilir"),
    ("kargo ücreti ne kadar tutuyor", "standart kargo ücretimiz 20 tl tutarındadır"),
    ("iade süresi kaç gün", "ürünü teslim aldıktan sonra 14 gün içinde iade edebilirsiniz"),
    ("nasıl iade edebilirim", "iade talebi oluşturup kargo kodu ile göndermeniz gerekir"),
    ("siparişi iptal etmek istiyorum", "hesabım sayfasından siparişinizi iptal edebilirsiniz"),
    ("kargom nerede", "siparişlerim bölümünden kargo takibi yapabilirsiniz")
]

print("--- UYGULAMA: İki farklı Chatbot modelini karşılaştıracağız. ---")
print("1. Basit Vektör Benzerliği Modeli (Encoder-Decoder'sız)")
print("2. Encoder-Decoder Modeli\n")
print("="*80)


# ############################################################################
# --- Model 1: Basit Benzerlik Modeli (Encoder-Decoder OLMADAN) ---
# ############################################################################
print("\n--- Model 1: Basit Benzerlik Modeli Başlatılıyor ---")

# 1. Veriyi hazırlama ve bir kelime vektör modeli (Word2Vec) eğitme
corpus = [gensim.utils.simple_preprocess(q) for q, a in qa_pairs] + \
         [gensim.utils.simple_preprocess(a) for q, a in qa_pairs]
w2v_model = gensim.models.Word2Vec(corpus, vector_size=100, window=2, min_count=1, workers=1)
print("Model 1 için kelime vektörleri (Word2Vec) eğitildi.")

def get_sentence_vector(sentence, model):
    """Bir cümlenin ortalama vektörünü hesaplar."""
    words = [word for word in gensim.utils.simple_preprocess(sentence) if word in model.wv]
    if not words:
        return np.zeros(model.vector_size)
    return np.mean(model.wv[words], axis=0)

# 2. Hazır cevapların vektörlerini önceden hesapla
questions = [pair[0] for pair in qa_pairs]
answers = [pair[1] for pair in qa_pairs]
answer_vectors = np.array([get_sentence_vector(ans, w2v_model) for ans in answers])

# 3. Basit Chatbot'un cevap verme fonksiyonu
def naive_chatbot_response(user_input, w2v_model, known_answers, answer_vectors):
    print(f"\n[Model 1] Kullanıcı Sorusu: '{user_input}'")
    
    # Kullanıcı girdisinin vektörünü hesapla
    input_vector = get_sentence_vector(user_input, w2v_model)
    
    # Kosinüs benzerliği ile en yakın cevabı bul
    # (sklearn.metrics.pairwise.cosine_similarity de kullanılabilir)
    similarities = np.dot(answer_vectors, input_vector) / (np.linalg.norm(answer_vectors, axis=1) * np.linalg.norm(input_vector))
    
    best_answer_index = np.argmax(similarities)
    print(f"[Model 1] En yakın bulduğu cevap: '{known_answers[best_answer_index]}'")
    
# Model 1'i test edelim
naive_chatbot_response("kargo ne kadar tutuyor", w2v_model, answers, answer_vectors)
naive_chatbot_response("ürünümü geri vermek istiyorum", w2v_model, answers, answer_vectors)

print("\nModel 1'in Yorumu: Bu model sadece en benzer hazır cevabı bulup getiriyor. Yeni bir cümle üretemiyor.")
print("="*80)


# ############################################################################
# --- Model 2: Encoder-Decoder Modeli ---
# ############################################################################
print("\n--- Model 2: Encoder-Decoder Modeli Başlatılıyor ---")
# Önceki kodumuzdaki Encoder-Decoder yapısını olduğu gibi kullanıyoruz.
# Sadece veri seti olarak Soru-Cevap çiftlerini veriyoruz.

SOS_token=0; EOS_token=1; device = torch.device("cpu"); hidden_size=128
class Lang:
    def __init__(self): self.word2index = {}; self.index2word = {0: "SOS", 1: "EOS"}; self.n_words = 2
    def addSentence(self, sentence):
        for word in sentence.split(' '): self.addWord(word)
    def addWord(self, word):
        if word not in self.word2index: self.word2index[word] = self.n_words; self.index2word[self.n_words] = word; self.n_words += 1
input_lang = Lang(); output_lang = Lang()
for q, a in qa_pairs: input_lang.addSentence(q); output_lang.addSentence(a)

class EncoderRNN(nn.Module):
    def __init__(self, i, h): super(EncoderRNN, self).__init__(); self.h=h; self.emb=nn.Embedding(i,h); self.gru=nn.GRU(h,h)
    def forward(self, inp, hid): emb=self.emb(inp).view(1,1,-1); out,hid=self.gru(emb,hid); return out,hid
    def initHidden(self): return torch.zeros(1, 1, self.h, device=device)
class DecoderRNN(nn.Module):
    def __init__(self, h, o): super(DecoderRNN, self).__init__(); self.h=h; self.emb=nn.Embedding(o,h); self.gru=nn.GRU(h,h); self.out=nn.Linear(h,o); self.softmax=nn.LogSoftmax(dim=1)
    def forward(self, inp, hid): out=self.emb(inp).view(1,1,-1); out=torch.relu(out); out,hid=self.gru(out,hid); out=self.softmax(self.out(out[0])); return out,hid

def train(inp_t, tar_t, enc, dec, enc_opt, dec_opt, crit):
    enc_h=enc.initHidden(); enc_opt.zero_grad(); dec_opt.zero_grad(); loss=0
    for ei in range(len(inp_t)): _, enc_h = enc(inp_t[ei], enc_h)
    dec_i=torch.tensor([[SOS_token]], device=device); dec_h=enc_h
    for di in range(len(tar_t)):
        dec_o, dec_h = dec(dec_i, dec_h); loss+=crit(dec_o, tar_t[di].unsqueeze(0)); dec_i=tar_t[di]
    loss.backward(); enc_opt.step(); dec_opt.step()
    return loss.item() / len(tar_t)

encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = DecoderRNN(hidden_size, output_lang.n_words).to(device)
encoder_optimizer = optim.SGD(encoder.parameters(), lr=0.01)
decoder_optimizer = optim.SGD(decoder.parameters(), lr=0.01)
criterion = nn.NLLLoss()

print("Encoder-Decoder modeli eğitime başlıyor...")
for i in range(5000): # Eğitim
    pair = random.choice(qa_pairs)
    inp_t = torch.LongTensor([input_lang.word2index[s] for s in pair[0].split(' ')]).to(device)
    tar_t = torch.LongTensor([output_lang.word2index[s] for s in pair[1].split(' ')] + [EOS_token]).to(device)
    train(inp_t, tar_t, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
print("Eğitim tamamlandı.")

def ed_chatbot_response(sentence, encoder, decoder):
    print(f"\n[Model 2] Kullanıcı Sorusu: '{sentence}'")
    with torch.no_grad():
        #inp_t=torch.LongTensor([input_lang.word2index[s] for s in sentence.split(' ')]).to(device)
        input_words = sentence.split(' ')
        input_indices = [input_lang.word2index[word] for word in input_words if word in input_lang.word2index]
        
        # Cümledeki hiçbir kelimeyi tanımıyorsak...
        if not input_indices:
            print(f"[Model 2] Üzgünüm, sorunuzdaki kelimeleri anlayamadım.")
            return
        
        inp_t = torch.LongTensor(input_indices).to(device)
        enc_h=encoder.initHidden()
        for ei in range(len(inp_t)): _, enc_h = encoder(inp_t[ei], enc_h)
        dec_i=torch.tensor([[SOS_token]], device=device); dec_h=enc_h
        decoded_words = []
        for di in range(15):
            dec_o, dec_h = decoder(dec_i, dec_h); _, topi = dec_o.data.topk(1)
            if topi.item() == EOS_token: break
            else: decoded_words.append(output_lang.index2word[topi.item()])
            dec_i = topi.squeeze().detach()
        response = ' '.join(decoded_words)
        print(f"[Model 2] Ürettiği Cevap: '{response}'")

# Model 2'yi aynı sorularla test edelim
ed_chatbot_response("kargo ne kadar tutuyor", encoder, decoder)
ed_chatbot_response("ürünümü geri vermek istiyorum", encoder, decoder)


# --- FİNAL KARŞILAŞTIRMA ---
print("\n" + "="*80)
print("--- FİNAL KARŞILAŞTIRMA VE YORUM ---")
print("\nSoru: 'kargo ücreti ne kadar'")
naive_chatbot_response("kargo ücreti ne kadar", w2v_model, answers, answer_vectors)
ed_chatbot_response("kargo ücreti ne kadar", encoder, decoder)

print("\nSoru: 'iptal işlemi nasıl yapılır'") # Veri setinde olmayan bir soru
naive_chatbot_response("iptal işlemi nasıl yapılır", w2v_model, answers, answer_vectors)
ed_chatbot_response("iptal işlemi nasıl yapılır", encoder, decoder)

print("\n>>> FARK <<<")
print("Basit model, 'iptal işlemi nasıl yapılır' sorusuna en benzer bulduğu 'iade talebi...' cevabını verdi. Çünkü sadece kelime benzerliğine bakıyor.")
print("Encoder-Decoder modeli ise, 'iptal' kelimesini anladı ve veri setindeki 'siparişinizi iptal edebilirsiniz' cümlesindeki yapıyı kullanarak daha doğru ve bağlama uygun bir cevap üretti.")
print("Encoder-Decoder yapısının farkı, sadece kelimeleri eşleştirmek değil, cümlenin anlamını bir bütün olarak kodlayıp bu anlamdan sıfırdan yeni bir cümle üretme yeteneğidir.")

--- UYGULAMA: İki farklı Chatbot modelini karşılaştıracağız. ---
1. Basit Vektör Benzerliği Modeli (Encoder-Decoder'sız)
2. Encoder-Decoder Modeli


--- Model 1: Basit Benzerlik Modeli Başlatılıyor ---
Model 1 için kelime vektörleri (Word2Vec) eğitildi.

[Model 1] Kullanıcı Sorusu: 'kargo ne kadar tutuyor'
[Model 1] En yakın bulduğu cevap: 'siparişlerim bölümünden kargo takibi yapabilirsiniz'

[Model 1] Kullanıcı Sorusu: 'ürünümü geri vermek istiyorum'
[Model 1] En yakın bulduğu cevap: 'iade talebi oluşturup kargo kodu ile göndermeniz gerekir'

Model 1'in Yorumu: Bu model sadece en benzer hazır cevabı bulup getiriyor. Yeni bir cümle üretemiyor.

--- Model 2: Encoder-Decoder Modeli Başlatılıyor ---
Encoder-Decoder modeli eğitime başlıyor...
Eğitim tamamlandı.

[Model 2] Kullanıcı Sorusu: 'kargo ne kadar tutuyor'
[Model 2] Ürettiği Cevap: 'standart kargo ücretimiz 20 tl tutarındadır'

[Model 2] Kullanıcı Sorusu: 'ürünümü geri vermek istiyorum'
[Model 2] Ürettiği Cevap: 'hesabım sayfasınd

In [6]:
qa_pairs = [
    ("ürün ne zaman kargoya verilir", "siparişiniz 1-3 iş günü içinde kargoya verilir"),
    ("kargo ücreti ne kadar tutuyor", "standart kargo ücretimiz 20 tl tutarındadır"),
    ("iade süresi kaç gün", "ürünü teslim aldıktan sonra 14 gün içinde iade edebilirsiniz"),
    ("nasıl iade edebilirim", "iade talebi oluşturup kargo kodu ile göndermeniz gerekir"),
    ("siparişi iptal etmek istiyorum", "hesabım sayfasından siparişinizi iptal edebilirsiniz"),
    ("kargom nerede", "siparişlerim bölümünden kargo takibi yapabilirsiniz")
]

In [7]:
ed_chatbot_response("turkcell", encoder, decoder)
ed_chatbot_response("turkcell ne kadar", encoder, decoder)
ed_chatbot_response("paketimi iptal etmek istiyorum", encoder, decoder)


[Model 2] Kullanıcı Sorusu: 'turkcell'
[Model 2] Üzgünüm, sorunuzdaki kelimeleri anlayamadım.

[Model 2] Kullanıcı Sorusu: 'turkcell ne kadar'
[Model 2] Ürettiği Cevap: 'standart kargo ücretimiz 20 tl tutarındadır'

[Model 2] Kullanıcı Sorusu: 'paketimi iptal etmek istiyorum'
[Model 2] Ürettiği Cevap: 'hesabım sayfasından siparişinizi iptal edebilirsiniz'


 ("ürün ne zaman kargoya verilir", "siparişiniz 1-3 iş günü içinde kargoya verilir"),
    ("kargo ücreti ne kadar tutuyor", "standart kargo ücretimiz 20 tl tutarındadır"),
    ("iade süresi kaç gün", "ürünü teslim aldıktan sonra 14 gün içinde iade edebilirsiniz"),
    ("nasıl iade edebilirim", "iade talebi oluşturup kargo kodu ile göndermeniz gerekir"),
    ("siparişi iptal etmek istiyorum", "hesabım sayfasından siparişinizi iptal edebilirsiniz"),
    ("kargom nerede", "siparişlerim bölümünden kargo takibi yapabilirsiniz")

**Kod Çıktısı ve Yorumu**

Bu kodu çalıştırdığınızda göreceğiniz fark çok nettir:

Bilinen Sorulara Yakın Sorular:

* Model 1 (Basit): "kargo ne kadar tutuyor" sorusu, veri setindeki "kargo ücreti ne kadar" sorusuna çok benzediği için doğru cevabı ("standart kargo ücretimiz...") bulup getirecektir. Bu, basit benzerlik aramasıdır.

* Model 2 (Encoder-Decoder): Bu soruya yine doğru cevabı üretecektir, ancak bunu cevabı bir yerden kopyalayarak değil, "kargo" ve "ne kadar" kelimelerini anlayıp, öğrendiği kalıplara göre kelime kelime (standart -> kargo -> ücretimiz -> ...) inşa ederek yapacaktır.

Bilinmeyen Ama Anlamı Yakın Sorular (Asıl Test):

* Model 1 (Basit): "iptal işlemi nasıl yapılır" gibi veri setinde olmayan bir soru sorduğunuzda, bu modelin kafası karışacaktır. Vektörel olarak en çok benzediğini düşündüğü, belki de içinde "nasıl" geçen "iade talebi oluşturup..." gibi alakasız bir hazır cevabı seçecektir. Çünkü onun için "iptal" kelimesinin anlamından çok, cümlenin genel vektörünün hangi hazır cevabın vektörüne yakın olduğu önemlidir.

* Model 2 (Encoder-Decoder): Bu model ise, Encoder sayesinde "iptal işlemi nasıl yapılır" cümlesindeki "iptal" ve "nasıl" anahtar kavramlarını anlayıp bir anlam vektörüne kodlar. Decoder, bu anlam vektörünü aldığında, eğitim verisinden öğrendiği "iptal" ile ilgili en olası yapının "hesabım sayfasından siparişinizi iptal edebilirsiniz" olduğunu bilir ve bu yapıya uygun yeni bir cümle üretir. Çıktısı, basit modelin aksine çok daha doğru ve bağlama uygun olacaktır.

Özetle fark şudur: Basit model bir kütüphanecidir, siz bir kitap adı söylediğinizde raftaki en benzer isimli kitabı bulup getirir. Encoder-Decoder ise bir yazardır, siz ona bir fikir verdiğinizde o fikri anlar ve o fikir hakkında sıfırdan yeni bir metin yazar. İşte bu üretken (generative) yetenek, Encoder-Decoder mimarisinin en büyük gücüdür.

#  UNK Token Kullanımı

In [8]:
import numpy as np
import gensim
import torch
import torch.nn as nn
import torch.optim as optim
import random
import warnings
warnings.filterwarnings("ignore")

# --- Ortak Veri Seti ---
qa_pairs = [
    ("ürün ne zaman kargoya verilir", "siparişiniz 1-3 iş günü içinde kargoya verilir"),
    ("kargo ücreti ne kadar tutuyor", "standart kargo ücretimiz 20 tl tutarındadır"),
    ("iade süresi kaç gün", "ürünü teslim aldıktan sonra 14 gün içinde iade edebilirsiniz"),
    ("nasıl iade edebilirim", "iade talebi oluşturup kargo kodu ile göndermeniz gerekir"),
    ("siparişi iptal etmek istiyorum", "hesabım sayfasından siparişinizi iptal edebilirsiniz"),
    ("kargom nerede", "siparişlerim bölümünden kargo takibi yapabilirsiniz"),
    ("turkcell ile bağlan hayata","turkcell ürünlerini turkcell.com.tr adresinden inceleyebilirsiniz"),
    ("kurt koyun yedi", "ilk ihtimal"),
    ("koyun kurt yedi", "ikinci ihtimal")
]

print("--- UYGULAMA: İki farklı Chatbot modelini karşılaştıracağız. ---")
print("1. Basit Vektör Benzerliği Modeli (Encoder-Decoder'sız)")
print("2. Encoder-Decoder Modeli (OOV Problemi için <UNK> Token ile Güçlendirildi)\n")
print("="*80)


# ############################################################################
# --- Model 1: Basit Benzerlik Modeli (Değişiklik yok) ---
# ############################################################################
print("\n--- Model 1: Basit Benzerlik Modeli Başlatılıyor ---")
# Bu modeldeki get_sentence_vector fonksiyonu zaten bilinmeyen kelimeleri atladığı için
# çökme yaşamaz, bu yüzden burada bir değişiklik yapmıyoruz.
corpus = [gensim.utils.simple_preprocess(q) for q, a in qa_pairs] + \
         [gensim.utils.simple_preprocess(a) for q, a in qa_pairs]
w2v_model = gensim.models.Word2Vec(corpus, vector_size=100, window=2, min_count=1, workers=1)
print("Model 1 için kelime vektörleri (Word2Vec) eğitildi.")
def get_sentence_vector(sentence, model):
    words = [word for word in gensim.utils.simple_preprocess(sentence) if word in model.wv]
    if not words: return np.zeros(model.vector_size)
    return np.mean(model.wv[words], axis=0)
questions = [pair[0] for pair in qa_pairs]; answers = [pair[1] for pair in qa_pairs]
answer_vectors = np.array([get_sentence_vector(ans, w2v_model) for ans in answers])
def naive_chatbot_response(user_input, w2v_model, known_answers, answer_vectors):
    print(f"\n[Model 1] Kullanıcı Sorusu: '{user_input}'")
    input_vector = get_sentence_vector(user_input, w2v_model)
    if np.all(input_vector == 0):
        print("[Model 1] Sorunuzdaki kelimeleri anlayamadım.")
        return
    similarities = np.dot(answer_vectors, input_vector) / (np.linalg.norm(answer_vectors, axis=1) * np.linalg.norm(input_vector))
    best_answer_index = np.argmax(similarities)
    print(f"[Model 1] En yakın bulduğu cevap: '{known_answers[best_answer_index]}'")
print("="*80)


# ############################################################################
# --- Model 2: Encoder-Decoder Modeli (<UNK> Token ile Düzeltildi) ---
# ############################################################################
print("\n--- Model 2: Encoder-Decoder Modeli Başlatılıyor ---")

SOS_token=0; EOS_token=1; UNK_token = 2 # <-- YENİ: UNK Token'ı tanımlandı
device = torch.device("cpu"); hidden_size=128

class Lang:
    def __init__(self):
        self.word2index = {}
        # YENİ: UNK token'ı sözlüğe eklendi
        self.index2word = {SOS_token: "SOS", EOS_token: "EOS", UNK_token: "<UNK>"}
        self.n_words = 3 # SOS, EOS ve UNK ile başla

    def addSentence(self, sentence):
        for word in sentence.split(' '): self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.index2word[self.n_words] = word
            self.n_words += 1

# Veri setini oluştur
input_lang = Lang(); output_lang = Lang()
for q, a in qa_pairs: input_lang.addSentence(q); output_lang.addSentence(a)

# YENİ: Tensör oluşturma fonksiyonları <UNK> token'ını kullanacak şekilde güncellendi
def sentence_to_tensor(lang, sentence):
    indexes = [lang.word2index.get(word, UNK_token) for word in sentence.split(' ')]
    return torch.LongTensor(indexes).to(device)

def pair_to_tensors(pair):
    input_tensor = sentence_to_tensor(input_lang, pair[0])
    target_tensor = sentence_to_tensor(output_lang, pair[1])
    target_tensor = torch.cat((target_tensor, torch.LongTensor([EOS_token]).to(device)))
    return (input_tensor, target_tensor)

# Modeller (Değişiklik yok)
class EncoderRNN(nn.Module):
    def __init__(self, i, h): super(EncoderRNN, self).__init__(); self.h=h; self.emb=nn.Embedding(i,h); self.gru=nn.GRU(h,h)
    def forward(self, inp, hid): emb=self.emb(inp).view(1,1,-1); out,hid=self.gru(emb,hid); return out,hid
    def initHidden(self): return torch.zeros(1, 1, self.h, device=device)
class DecoderRNN(nn.Module):
    def __init__(self, h, o): super(DecoderRNN, self).__init__(); self.h=h; self.emb=nn.Embedding(o,h); self.gru=nn.GRU(h,h); self.out=nn.Linear(h,o); self.softmax=nn.LogSoftmax(dim=1)
    def forward(self, inp, hid): out=self.emb(inp).view(1,1,-1); out=torch.relu(out); out,hid=self.gru(out,hid); out=self.softmax(self.out(out[0])); return out,hid

# Eğitim fonksiyonu (Değişiklik yok)
def train(inp_t, tar_t, enc, dec, enc_opt, dec_opt, crit):
    enc_h=enc.initHidden(); enc_opt.zero_grad(); dec_opt.zero_grad(); loss=0
    for ei in range(len(inp_t)): _, enc_h = enc(inp_t[ei], enc_h)
    dec_i=torch.tensor([[SOS_token]], device=device); dec_h=enc_h
    for di in range(len(tar_t)):
        dec_o, dec_h = dec(dec_i, dec_h); loss+=crit(dec_o, tar_t[di].unsqueeze(0)); dec_i=tar_t[di]
    loss.backward(); enc_opt.step(); dec_opt.step()

# Eğitim
encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = DecoderRNN(hidden_size, output_lang.n_words).to(device)
encoder_optimizer = optim.SGD(encoder.parameters(), lr=0.01)
decoder_optimizer = optim.SGD(decoder.parameters(), lr=0.01)
criterion = nn.NLLLoss()

print("Encoder-Decoder modeli eğitime başlıyor...")
for i in range(5000):
    pair = random.choice(qa_pairs)
    input_tensor, target_tensor = pair_to_tensors(pair)
    train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
print("Eğitim tamamlandı.")

# Test fonksiyonu (Artık <UNK> kullanıyor)
def ed_chatbot_response(sentence, encoder, decoder):
    print(f"\n[Model 2] Kullanıcı Sorusu: '{sentence}'")
    with torch.no_grad():
        input_tensor = sentence_to_tensor(input_lang, sentence)
        enc_h=encoder.initHidden()
        for ei in range(len(input_tensor)): _, enc_h = encoder(input_tensor[ei], enc_h)
        dec_i=torch.tensor([[SOS_token]], device=device); dec_h=enc_h
        decoded_words = []
        for di in range(15):
            dec_o, dec_h = decoder(dec_i, dec_h); _, topi = dec_o.data.topk(1)
            if topi.item() == EOS_token: break
            # <UNK> token'ı üretirse bunu atlayabiliriz veya gösterebiliriz
            if topi.item() == UNK_token:
                decoded_words.append("<BILINMIYOR>")
            else:
                decoded_words.append(output_lang.index2word[topi.item()])
            dec_i = topi.squeeze().detach()
        response = ' '.join(decoded_words)
        print(f"[Model 2] Ürettiği Cevap: '{response}'")

# --- FİNAL KARŞILAŞTIRMA ---
print("\n" + "="*80)
print("--- FİNAL KARŞILAŞTIRMA VE YORUM ---")

# Test cümlesi (Bilinmeyen kelimeler içeriyor: 'iadesi' ve 'yapılıyor')
test_cumlesi = "iade iadesi nasıl yapılıyor"
print(f"\nTest Edilen Soru: '{test_cumlesi}'\n")

# Model 1'in cevabı
naive_chatbot_response(test_cumlesi, w2v_model, answers, answer_vectors)

# Model 2'nin cevabı
ed_chatbot_response(test_cumlesi, encoder, decoder)

print("\n>>> FARK <<<")
print("Basit model, bilinmeyen kelimeleri atlayarak en benzer cümleyi bulmaya çalıştı.")
print("Encoder-Decoder modeli ise, bilinmeyen 'iadesi' ve 'yapılıyor' kelimelerini <UNK> olarak işledi.")
print("Ancak cümlenin geri kalanındaki 'iade' ve 'nasıl' kelimelerini anladığı için doğru bağlamdaki cevabı ('iade talebi...') üretebildi. Bu, çok daha sağlam bir yöntemdir.")

--- UYGULAMA: İki farklı Chatbot modelini karşılaştıracağız. ---
1. Basit Vektör Benzerliği Modeli (Encoder-Decoder'sız)
2. Encoder-Decoder Modeli (OOV Problemi için <UNK> Token ile Güçlendirildi)


--- Model 1: Basit Benzerlik Modeli Başlatılıyor ---
Model 1 için kelime vektörleri (Word2Vec) eğitildi.

--- Model 2: Encoder-Decoder Modeli Başlatılıyor ---
Encoder-Decoder modeli eğitime başlıyor...
Eğitim tamamlandı.

--- FİNAL KARŞILAŞTIRMA VE YORUM ---

Test Edilen Soru: 'iade iadesi nasıl yapılıyor'


[Model 1] Kullanıcı Sorusu: 'iade iadesi nasıl yapılıyor'
[Model 1] En yakın bulduğu cevap: 'ürünü teslim aldıktan sonra 14 gün içinde iade edebilirsiniz'

[Model 2] Kullanıcı Sorusu: 'iade iadesi nasıl yapılıyor'
[Model 2] Ürettiği Cevap: 'iade talebi oluşturup kargo kodu ile göndermeniz gerekir'

>>> FARK <<<
Basit model, bilinmeyen kelimeleri atlayarak en benzer cümleyi bulmaya çalıştı.
Encoder-Decoder modeli ise, bilinmeyen 'iadesi' ve 'yapılıyor' kelimelerini <UNK> olarak işledi.
A

In [9]:
# Test cümlesi (Bilinmeyen kelimeler içeriyor: 'iadesi' ve 'yapılıyor')
test_cumlesi = "kurt koyun oynadı"
print(f"\nTest Edilen Soru: '{test_cumlesi}'\n")

# Model 1'in cevabı
naive_chatbot_response(test_cumlesi, w2v_model, answers, answer_vectors)

# Model 2'nin cevabı
ed_chatbot_response(test_cumlesi, encoder, decoder)

# Test cümlesi (Bilinmeyen kelimeler içeriyor: 'iadesi' ve 'yapılıyor')
test_cumlesi = "koyun kurt oynadı"
print(f"\nTest Edilen Soru: '{test_cumlesi}'\n")

# Model 1'in cevabı
naive_chatbot_response(test_cumlesi, w2v_model, answers, answer_vectors)

# Model 2'nin cevabı
ed_chatbot_response(test_cumlesi, encoder, decoder)


Test Edilen Soru: 'kurt koyun oynadı'


[Model 1] Kullanıcı Sorusu: 'kurt koyun oynadı'
[Model 1] En yakın bulduğu cevap: 'hesabım sayfasından siparişinizi iptal edebilirsiniz'

[Model 2] Kullanıcı Sorusu: 'kurt koyun oynadı'
[Model 2] Ürettiği Cevap: 'ilk ihtimal'

Test Edilen Soru: 'koyun kurt oynadı'


[Model 1] Kullanıcı Sorusu: 'koyun kurt oynadı'
[Model 1] En yakın bulduğu cevap: 'hesabım sayfasından siparişinizi iptal edebilirsiniz'

[Model 2] Kullanıcı Sorusu: 'koyun kurt oynadı'
[Model 2] Ürettiği Cevap: 'ikinci ihtimal'


# Cümle Üretimi

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import random

# --- Adım 1: Veri Seti ve Geliştirilmiş Lang Sınıfı ---
print("--- Adım 1: Veri Hazırlığı (<UNK> Token ile Güçlendirildi) ---")

SOS_token = 0  # Start of Sentence
EOS_token = 1  # End of Sentence
UNK_token = 2  # YENİ: Unknown token tanımlandı

pairs = [
    ("you are a student", "are you a student"),
    ("he is a doctor", "is he a doctor"),
    ("they are engineers", "are they engineers"),
    ("the cat is black", "is the cat black"),
    ("we can go now", "can we go now"),
    ("she will be here", "will she be here"),
    ("this is my book", "is this my book"),
    ("you are happy", "are you happy")
]

# DÜZELTME: Lang sınıfı artık UNK token'ını içeriyor
class Lang:
    def __init__(self):
        self.word2index = {}
        self.index2word = {SOS_token: "SOS", EOS_token: "EOS", UNK_token: "<UNK>"}
        self.n_words = 3  # SOS, EOS ve UNK ile başla

    def addSentence(self, sentence):
        for word in sentence.split(' '): self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.index2word[self.n_words] = word
            self.n_words += 1

input_lang = Lang(); output_lang = Lang()
for pair in pairs: input_lang.addSentence(pair[0]); output_lang.addSentence(pair[1])

# DÜZELTME: Bu fonksiyon artık bilinmeyen kelimelerle başa çıkabiliyor
def sentence_to_tensor(lang, sentence):
    """Bir cümleyi, OOV kelimeler için UNK token'ı kullanarak tensöre çevirir."""
    indexes = [lang.word2index.get(word, UNK_token) for word in sentence.split(' ')]
    indexes.append(EOS_token)
    return torch.LongTensor(indexes)

print("Veri seti ve <UNK> destekli sözlükler oluşturuldu.\n")
print("="*60)

# --- Adım 2: Encoder-Decoder Modelleri (Yapısal Değişiklik Yok) ---
print("\n--- Adım 2: Encoder ve Decoder Modelleri ---")
device = torch.device("cpu"); hidden_size = 128
class EncoderRNN(nn.Module):
    def __init__(self, i, h): super(EncoderRNN, self).__init__(); self.emb=nn.Embedding(i,h); self.gru=nn.GRU(h,h)
    def forward(self, inp, hid): emb=self.emb(inp).view(1,1,-1); _,hid=self.gru(emb,hid); return hid
    def initHidden(self): return torch.zeros(1, 1, hidden_size, device=device)
class DecoderRNN(nn.Module):
    def __init__(self, h, o): super(DecoderRNN, self).__init__(); self.emb=nn.Embedding(o,h); self.gru=nn.GRU(h,h); self.out=nn.Linear(h,o); self.softmax=nn.LogSoftmax(dim=1)
    def forward(self, inp, hid): out=self.emb(inp).view(1,1,-1); out=torch.relu(out); out,hid=self.gru(out,hid); out=self.softmax(self.out(out[0])); return out,hid

# --- Adım 3: Eğitim (Değişiklik yok, ama artık UNK token'ını da öğrenebilir) ---
print("\n--- Adım 3: Modelin Cümle Yapısını Öğrenmesi ---")

def train(inp_t, tar_t, encoder, decoder, enc_opt, dec_opt, criterion):
    enc_h = encoder.initHidden()
    enc_opt.zero_grad(); dec_opt.zero_grad()
    loss = 0
    for ei in range(len(inp_t)):
        enc_h = encoder(inp_t[ei].unsqueeze(0), enc_h)
    
    dec_i = torch.tensor([[SOS_token]], device=device)
    dec_h = enc_h
    
    for di in range(len(tar_t)):
        dec_o, dec_h = decoder(dec_i, dec_h)
        loss += criterion(dec_o, tar_t[di].unsqueeze(0))
        dec_i = tar_t[di]
        if dec_i.item() == EOS_token: break
            
    loss.backward()
    enc_opt.step(); dec_opt.step()

encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = DecoderRNN(hidden_size, output_lang.n_words).to(device)
encoder_optimizer = optim.Adam(encoder.parameters(), lr=0.001)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=0.001)
criterion = nn.NLLLoss()

n_iters = 10000
print(f"{n_iters} iterasyonluk eğitim başlıyor...")

for iter in range(1, n_iters + 1):
    pair = random.choice(pairs)
    input_tensor = sentence_to_tensor(input_lang, pair[0])
    target_tensor = sentence_to_tensor(output_lang, pair[1])
    train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)

print("Eğitim tamamlandı.")
print("="*60)

# --- Adım 4: Cümle Kuran Modeli Test Etme ---
print("\n--- Adım 4: Hata Ayıklanmış Modelin Test Edilmesi ---")

def generate_question(sentence, encoder, decoder):
    print(f"\n>>> Girdi Cümlesi: '{sentence}'")
    with torch.no_grad():
        # Düzeltilmiş fonksiyonumuz sayesinde bu satır artık çökmeyecek
        input_tensor = sentence_to_tensor(input_lang, sentence)
        
        enc_h = encoder.initHidden()
        for ei in range(len(input_tensor)):
            enc_h = encoder(input_tensor[ei].unsqueeze(0), enc_h)

        dec_i = torch.tensor([[SOS_token]], device=device)
        dec_h = enc_h
        decoded_words = []
        
        print("Modelin Adım Adım Ürettiği Çıktı:")
        for di in range(10):
            dec_o, dec_h = decoder(dec_i, dec_h)
            _, topi = dec_o.data.topk(1)
            dec_i = topi.squeeze().detach().unsqueeze(0)
            
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                word = output_lang.index2word[topi.item()]
                decoded_words.append(word)
                print(f"   -> Adım {di+1}: '{word}' üretildi.")

        return ' '.join(decoded_words)

# Eğitim verisinden bir örnekle test
generate_question("he is a doctor", encoder, decoder)

# HATA VEREN CÜMLEYİ TEKRAR TEST ETME
generate_question("the dog is happy", encoder, decoder)

# Başka bir bilinmeyen kelime içeren cümle
generate_question("we are teachers", encoder, decoder)

--- Adım 1: Veri Hazırlığı (<UNK> Token ile Güçlendirildi) ---
Veri seti ve <UNK> destekli sözlükler oluşturuldu.


--- Adım 2: Encoder ve Decoder Modelleri ---

--- Adım 3: Modelin Cümle Yapısını Öğrenmesi ---
10000 iterasyonluk eğitim başlıyor...
Eğitim tamamlandı.

--- Adım 4: Hata Ayıklanmış Modelin Test Edilmesi ---

>>> Girdi Cümlesi: 'he is a doctor'
Modelin Adım Adım Ürettiği Çıktı:
   -> Adım 1: 'is' üretildi.
   -> Adım 2: 'he' üretildi.
   -> Adım 3: 'a' üretildi.
   -> Adım 4: 'doctor' üretildi.

>>> Girdi Cümlesi: 'the dog is happy'
Modelin Adım Adım Ürettiği Çıktı:
   -> Adım 1: 'is' üretildi.
   -> Adım 2: 'the' üretildi.
   -> Adım 3: 'cat' üretildi.
   -> Adım 4: 'black' üretildi.

>>> Girdi Cümlesi: 'we are teachers'
Modelin Adım Adım Ürettiği Çıktı:
   -> Adım 1: 'are' üretildi.
   -> Adım 2: 'you' üretildi.
   -> Adım 3: 'happy' üretildi.


'are you happy <EOS>'

In [11]:
pairs = [
    ("you are a student", "are you a student"),
    ("he is a doctor", "is he a doctor"),
    ("they are engineers", "are they engineers"),
    ("the cat is black", "is the cat black"),
    ("we can go now", "can we go now"),
    ("she will be here", "will she be here"),
    ("this is my book", "is this my book"),
    ("you are happy", "are you happy")
]

# Decoder Only, Encoder Only , Decoder - Encoder Yapıları

In [12]:
# Gerekli kütüphaneleri ve özellikle pipeline aracını içe aktarıyoruz.
# pipeline, karmaşık kodlar yazmadan modelleri hızlıca kullanmamızı sağlar.
from transformers import pipeline
import warnings
warnings.filterwarnings("ignore")

# ####################################################################################
# --- BÖLÜM 1: ENCODER-ONLY MİMARİSİ (BERT) ---
# Görev: Metni Anlamak ve Bağlamdan Çıkarım Yapmak
# ####################################################################################
print("="*80)
print("--- BÖLÜM 1: ENCODER-ONLY (BERT ile Maske Doldurma) ---")
print("GÖREV: Cümledeki eksik kelimeyi, cümlenin hem başını hem de sonunu dikkate alarak tahmin etmek.\n")

# 'fill-mask' görevi için bir pipeline oluşturuyoruz.
# Model olarak Türkçe için çok yaygın kullanılan bir BERT modeli seçiyoruz.
mask_filler = pipeline("fill-mask", model="dbmdz/bert-base-turkish-cased")

# BERT'e ortasında [MASK] token'ı olan bir cümle veriyoruz.
masked_sentence = "İstanbul, Türkiye'nin en [MASK] şehridir."
print(f"Girdi Cümlesi: {masked_sentence}\n")

print("BERT'in Tahminleri:")
# Modelden en olası 5 tahmini istiyoruz.
predictions = mask_filler(masked_sentence, top_k=5)
for pred in predictions:
    print(f"  -> Tahmin: '{pred['token_str']}', Skor: {pred['score']:.4f}")

print("\n>>> YORUM: BERT (Encoder-Only), 'İstanbul' ve 'şehridir' kelimelerine aynı anda bakarak (çift yönlü)")
print("boşluğa en uygun kelimenin 'kalabalık' veya 'büyük' olduğuna karar verdi. Bu, derin bir anlama yeteneğidir.")


# ####################################################################################
# --- BÖLÜM 2: DECODER-ONLY MİMARİSİ (GPT) ---
# Görev: Sıfırdan Metin Üretmek ve Devam Ettirmek
# ####################################################################################
print("\n" + "="*80)
print("--- BÖLÜM 2: DECODER-ONLY (GPT ile Metin Üretme) ---")
print("GÖREV: Verilen bir başlangıç metnini (prompt) mantıklı ve tutarlı bir şekilde devam ettirmek.\n")

# 'text-generation' görevi için bir pipeline oluşturuyoruz.
# Türkçe için eğitilmiş bir GPT-2 modeli kullanıyoruz.
text_generator = pipeline("text-generation", model="ytu-ce-cosmos/turkish-gpt2")

# GPT'ye bir başlangıç metni veriyoruz.
prompt = "Yapay zeka, son yıllarda en çok konuşulan konulardan biri haline geldi. Özellikle"
print(f"Başlangıç Metni (Prompt): '{prompt}...'\n")

print("GPT'nin Ürettiği Devam Metni:")
# Modelden bu metni 100 karakterlik bir metinle devam ettirmesini istiyoruz.
generated_text = text_generator(prompt, max_length=100, num_return_sequences=1)
print(generated_text[0]['generated_text'])

print("\n>>> YORUM: GPT (Decoder-Only), sadece kendisinden önce gelen kelimelere bakarak (tek yönlü)")
print("adım adım yeni kelimeler üretti ve başlangıç metnini dilbilgisi kurallarına uygun bir şekilde devam ettirdi. Bu, saf bir üretme yeteneğidir.")


# ####################################################################################
# --- BÖLÜM 3: ENCODER-DECODER MİMARİSİ (T5) ---
# Görev: Bir Metin Formatını Başka Bir Formata Dönüştürmek
# ####################################################################################
print("\n" + "="*80)
print("--- BÖLÜM 3: ENCODER-DECODER (T5 ile Özetleme) ---")
print("GÖREV: Uzun bir metni okuyup anladıktan sonra, bu anlayışı kullanarak kısa ve yeni bir özet metni üretmek.\n")

# 'summarization' görevi için bir pipeline oluşturuyoruz.
# T5, bu tür dönüşüm görevleri için tasarlanmış bir Encoder-Decoder modelidir.
summarizer = pipeline("summarization", model="google-t5/t5-base")

# Özetlenecek uzun bir metin hazırlayalım.
long_text = """
The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France. 
It is named after the engineer Gustave Eiffel, whose company designed and built the tower. 
Locally nicknamed 'La dame de fer' (French for 'Iron Lady'), the structure was built from 1887 to 1889 
as the centerpiece of the 1889 World's Fair. Although initially criticized by some of France's leading 
artists and intellectuals for its design, it has since become a global cultural icon of France and 
one of the most recognizable structures in the world.
"""
print(f"Özetlenecek Orijinal Metin:\n{long_text}\n")

print("T5 Modelinin Ürettiği Özet:")
# Modelden metnin özetini oluşturmasını istiyoruz.
summary = summarizer(long_text, max_length=50, min_length=10, do_sample=False)
print(summary[0]['summary_text'])

print("\n>>> YORUM: T5 (Encoder-Decoder), önce Encoder kısmı ile tüm metni okuyup anladı.")
print("Ardından Decoder kısmı, bu genel anlayıştan yola çıkarak metnin ana fikrini içeren tamamen yeni ve kısa bir cümle üretti. Bu, bir formattan diğerine dönüşümdür.")

2025-07-06 06:34:23.985906: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751783664.240461      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751783664.311876      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


--- BÖLÜM 1: ENCODER-ONLY (BERT ile Maske Doldurma) ---
GÖREV: Cümledeki eksik kelimeyi, cümlenin hem başını hem de sonunu dikkate alarak tahmin etmek.



config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


Özetlenecek Orijinal Metin:

The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France. 
It is named after the engineer Gustave Eiffel, whose company designed and built the tower. 
Locally nicknamed 'La dame de fer' (French for 'Iron Lady'), the structure was built from 1887 to 1889 
as the centerpiece of the 1889 World's Fair. Although initially criticized by some of France's leading 
artists and intellectuals for its design, it has since become a global cultural icon of France and 
one of the most recognizable structures in the world.


T5 Modelinin Ürettiği Özet:
the tower is named after the engineer Gustave Eiffel, whose company designed and built the tower . locally nicknamed 'la dame de fer' (french for 'iron lady'), it

>>> YORUM: T5 (Encoder-Decoder), önce Encoder kısmı ile tüm metni okuyup anladı.
Ardından Decoder kısmı, bu genel anlayıştan yola çıkarak metnin ana fikrini içeren tamamen yeni ve kısa bir cümle üretti. Bu, bir formattan diğerine dönü

# Uçtan Uca Konular

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
from sklearn.metrics.pairwise import cosine_similarity

# --- Genel Kurulum ---
# Bu kurulum tüm aşamalarda kullanılacak
device = torch.device("cpu")
SOS_token = 0; EOS_token = 1

# Veri setimiz: İngilizce'den Türkçe'ye çeviri çiftleri
pairs = [
    ("i am a student", "ben bir öğrenciyim"),
    ("he is a doctor", "o bir doktor"),
    ("they are engineers", "onlar mühendis"),
    ("i run fast", "ben hızlı koşarım"),
    ("she reads a book", "o bir kitap okur")
]

# Kelime dağarcığı oluşturmak için Lang sınıfı
class Lang:
    def __init__(self):
        self.word2index = {}; self.index2word = {0: "SOS", 1: "EOS"}; self.n_words = 2
    def addSentence(self, sentence):
        for word in sentence.split(' '): self.addWord(word)
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words; self.index2word[self.n_words] = word; self.n_words += 1
            
input_lang = Lang(); output_lang = Lang()
for pair in pairs:
    input_lang.addSentence(pair[0]); output_lang.addSentence(pair[1])

# Hiperparametreler
embedding_dim = 64
hidden_size = 64

# --- AŞAMA 1: Sadece Embedding - "Anlam Torbası" Yaklaşımı ---
print("="*80)
print("--- AŞAMA 1: Sadece Embedding ile Anlam Yakalamaya Çalışmak (Başarısız Olacak) ---")
print("Fikir: Bir cümlenin anlamı, içindeki kelimelerin anlamlarının ortalamasıdır.\n")

# Kelime vektörlerini oluşturmak için bir embedding katmanı
embedding_layer = nn.Embedding(input_lang.n_words, embedding_dim)

def get_avg_vector(sentence, lang, layer):
    """Cümlenin ortalama embedding vektörünü hesaplar."""
    with torch.no_grad():
        indices = torch.LongTensor([lang.word2index[w] for w in sentence.split(' ')])
        embeddings = layer(indices)
        return embeddings.mean(dim=0)

# Veri setindeki her cümlenin ortalama vektörünü hesaplayalım
input_vectors = [get_avg_vector(pair[0], input_lang, embedding_layer) for pair in pairs]
target_vectors = [get_avg_vector(pair[1], output_lang, embedding_layer) for pair in pairs]

def translate_with_averaging(sentence, input_vectors, target_sentences):
    """Girdi cümlesine en çok benzeyen hedef cümleyi bulur."""
    input_vec = get_avg_vector(sentence, input_lang, embedding_layer).numpy().reshape(1, -1)
    # Numpy'a çevirerek benzerlik hesapla
    target_vecs_np = torch.stack(target_vectors).numpy()
    similarities = cosine_similarity(input_vec, target_vecs_np)
    best_match_index = np.argmax(similarities)
    return target_sentences[best_match_index]

# Test edelim
test_sentence_1 = "he is a student" # "he" ve "student" kelimelerini biliyor
print(f"Girdi: '{test_sentence_1}'")
translation = translate_with_averaging(test_sentence_1, input_vectors, [p[1] for p in pairs])
print(f"Çıktı (En Yakın Cümle): '{translation}'")

print("\n>>> KAZANIM/PROBLEM:")
print("Bu yöntem, kelimelerin 'anlam torbası' gibi davranır. Kelime sırasını ve grameri tamamen yok sayar.")
print("Bu yüzden 'he is a student' gibi bir cümleyi doğru ayırt edemez, çünkü hem 'he is a doctor'")
print("hem de 'i am a student' cümlelerine anlamsal olarak benzerdir. Dizilim bilgisini kodlayacak bir yapıya ihtiyacımız var.")
print("="*80)


# --- AŞAMA 2: Encoder'ı Eklemek - Sıralı Anlamı Yoğunlaştırma ---
print("\n--- AŞAMA 2: Encoder ile Cümle Dizilimini Anlamak ---")
print("Fikir: Bir RNN (GRU), cümleyi kelime kelime okuyarak dizilim bilgisini tek bir 'bağlam vektöründe' özetler.\n")

class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        _, hidden = self.gru(embedded, hidden)
        return hidden
    def initHidden(self): return torch.zeros(1, 1, hidden_size, device=device)

encoder = Encoder(input_lang.n_words, hidden_size)

def encode_sentence(sentence, lang, encoder_model):
    """Bir cümleyi tek bir bağlam vektörüne kodlar."""
    with torch.no_grad():
        indices = torch.LongTensor([lang.word2index[w] for w in sentence.split(' ')])
        hidden = encoder_model.initHidden()
        for i in range(len(indices)):
            # Her kelime sırayla işlenir ve gizli durum güncellenir
            hidden = encoder_model(indices[i].unsqueeze(0), hidden)
        return hidden

# Test edelim
test_sentence_2 = "i run fast"
context_vector = encode_sentence(test_sentence_2, input_lang, encoder)
print(f"Girdi: '{test_sentence_2}'")
print(f"Encoder'ın ürettiği 'Anlam Özeti' (Context Vector) boyutu: {context_vector.shape}")
print("Bu vektör, cümlenin hem kelimelerini hem de onların sırasını içerir.")

print("\n>>> KAZANIM/PROBLEM:")
print("Artık elimizde cümlenin sıralı anlamını taşıyan güçlü bir özet var. Ancak bu özetten yola çıkarak")
print("nasıl kelime kelime yeni bir cümle 'üretebiliriz'? Bunun için bir üreticiye, yani Decoder'a ihtiyacımız var.")
print("="*80)


# --- AŞAMA 3: Encoder + Decoder - Anlama ve Sıfırdan Üretme ---
print("\n--- AŞAMA 3: Encoder-Decoder ile Üretken Çeviri ---")
print("Fikir: Encoder'ın ürettiği 'anlam özeti', Decoder için bir başlangıç noktasıdır. Decoder, bu özetle başlar ve adım adım yeni bir cümle kurar.\n")

class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.log_softmax = nn.LogSoftmax(dim=1)
    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1); output = torch.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.log_softmax(self.out(output[0])); return output, hidden

# Eğitim
encoder = Encoder(input_lang.n_words, hidden_size)
decoder = Decoder(hidden_size, output_lang.n_words)
enc_optimizer = optim.SGD(encoder.parameters(), lr=0.01)
dec_optimizer = optim.SGD(decoder.parameters(), lr=0.01)
criterion = nn.NLLLoss()

print("Encoder-Decoder modeli eğitiliyor...")
for i in range(10000): # Eğitim iterasyonları
    pair = random.choice(pairs)
    input_tensor = torch.LongTensor([input_lang.word2index[s] for s in pair[0].split(' ')])
    target_tensor = torch.LongTensor([output_lang.word2index[s] for s in pair[1].split(' ')] + [EOS_token])
    
    # Gerçek eğitim döngüsü (sessiz)
    enc_hidden = encoder.initHidden()
    enc_optimizer.zero_grad(); dec_optimizer.zero_grad()
    loss = 0
    for ei in range(len(input_tensor)): enc_hidden = encoder(input_tensor[ei].unsqueeze(0), enc_hidden)
    dec_input = torch.tensor([[SOS_token]]); dec_hidden = enc_hidden
    for di in range(len(target_tensor)):
        dec_output, dec_hidden = decoder(dec_input, dec_hidden)
        loss += criterion(dec_output, target_tensor[di].unsqueeze(0)); dec_input = target_tensor[di]
    loss.backward(); enc_optimizer.step(); dec_optimizer.step()
print("Eğitim tamamlandı.\n")

# Test ve Adım Adım Üretimi Gösterme
def translate_with_encoder_decoder(sentence):
    print(f"--- Girdi: '{sentence}' ---")
    with torch.no_grad():
        input_tensor = torch.LongTensor([input_lang.word2index[s] for s in sentence.split(' ')])
        
        # Encoder Aşaması
        enc_hidden = encoder.initHidden()
        for ei in range(len(input_tensor)):
            enc_hidden = encoder(input_tensor[ei].unsqueeze(0), enc_hidden)
        print("1. Encoder girdi cümlesini okudu ve bağlam vektörünü üretti.")

        # Decoder Aşaması
        dec_input = torch.tensor([[SOS_token]])
        dec_hidden = enc_hidden
        decoded_words = []
        print("2. Decoder, bağlam vektörüyle üretime başlıyor:")
        for di in range(10): # Maksimum 10 kelime
            dec_output, dec_hidden = decoder(dec_input, dec_hidden)
            topv, topi = dec_output.data.topk(1)
            word = output_lang.index2word[topi.item()]
            print(f"   -> Adım {di+1}: '{word}' kelimesi üretildi.")
            if topi.item() == EOS_token: break
            decoded_words.append(word)
            dec_input = topi.squeeze().detach().unsqueeze(0)
        
        print(f"Nihai Çıktı: '{' '.join(decoded_words)}'")

translate_with_encoder_decoder("i run fast")
translate_with_encoder_decoder("she is a doctor")

print("\n>>> KAZANIM/PROBLEM:")
print("Artık modelimiz sadece en yakın cümleyi bulmuyor, bir cümlenin sıralı anlamını (Encoder ile)")
print("kavrayıp bu anlayıştan yola çıkarak kelime kelime, gramer kurallarına uygun yeni bir cümle (Decoder ile) inşa ediyor.")
print("Bu, dil işlemede basit benzerlikten gerçek üretkenliğe geçiştir.")

--- AŞAMA 1: Sadece Embedding ile Anlam Yakalamaya Çalışmak (Başarısız Olacak) ---
Fikir: Bir cümlenin anlamı, içindeki kelimelerin anlamlarının ortalamasıdır.

Girdi: 'he is a student'
Çıktı (En Yakın Cümle): 'o bir doktor'

>>> KAZANIM/PROBLEM:
Bu yöntem, kelimelerin 'anlam torbası' gibi davranır. Kelime sırasını ve grameri tamamen yok sayar.
Bu yüzden 'he is a student' gibi bir cümleyi doğru ayırt edemez, çünkü hem 'he is a doctor'
hem de 'i am a student' cümlelerine anlamsal olarak benzerdir. Dizilim bilgisini kodlayacak bir yapıya ihtiyacımız var.

--- AŞAMA 2: Encoder ile Cümle Dizilimini Anlamak ---
Fikir: Bir RNN (GRU), cümleyi kelime kelime okuyarak dizilim bilgisini tek bir 'bağlam vektöründe' özetler.

Girdi: 'i run fast'
Encoder'ın ürettiği 'Anlam Özeti' (Context Vector) boyutu: torch.Size([1, 1, 64])
Bu vektör, cümlenin hem kelimelerini hem de onların sırasını içerir.

>>> KAZANIM/PROBLEM:
Artık elimizde cümlenin sıralı anlamını taşıyan güçlü bir özet var. Ancak bu özetten

# Embedding, Attention ve Decoder ile Cümle Üretimi

Bu, öğrendiğimiz tüm kavramları (Embedding, Attention, Decoder) bir araya getirerek, modern bir sohbet robotunun temel "düşünme" sürecini gösteren, uçtan uca ve öğretici bir kod örneği olacak.

Bu kod, bir soruya cevap üretirken her bir bileşenin ne işe yaradığını ve bir sonraki aşamaya nasıl bir "kazanım" sağladığını adım adım ara çıktılarla gösterecek şekilde tasarlanmıştır.

**Senaryo ve Anlatım Akışı**

Bir soru-cevap chatbot'u yapacağız. Hikayemiz üç aşamadan oluşacak:

* Aşama 1: Embedding - Anlam Dünyasına Giriş: Bir cümlenin kelimelerini, makinenin anlayabileceği sayılara (vektörlere) nasıl dönüştürdüğümüzü göreceğiz.

* Aşama 2: Attention - Akıllıca Odaklanma: Cevap üretirken, Decoder'ın orijinal sorudaki hangi kelimelere "daha fazla dikkat etmesi" gerektiğini nasıl anladığını görselleştireceğiz.

* Aşama 3: Decoder - Cümle Kurma Sanatı: Bu odaklanmış bilgiyi alan Decoder'ın, nasıl adım adım tutarlı ve bağlama uygun bir cevap cümlesi inşa ettiğini izleyeceğiz.

* Mimari: Bu örnek için, Attention mekanizmasının rolünü en net şekilde gösterebilen Attention'lı Encoder-Decoder (RNN/GRU tabanlı) yapısını kullanacağız.

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import torch.nn.functional as F
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import warnings
warnings.filterwarnings("ignore")

# --- Adım 0: Kurulum ve Veri Hazırlığı ---
device = torch.device("cpu")
SOS_token = 0; EOS_token = 1; UNK_token = 2

class Lang:
    def __init__(self):
        self.word2index = {}; self.index2word = {0: "SOS", 1: "EOS", 2: "<UNK>"}; self.n_words = 3
    def addSentence(self, sentence):
        for word in sentence.split(' '): self.addWord(word)
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words; self.index2word[self.n_words] = word; self.n_words += 1

def sentence_to_tensor(lang, sentence):
    indexes = [lang.word2index.get(word, UNK_token) for word in sentence.split(' ')]
    indexes.append(EOS_token)
    return torch.LongTensor(indexes).view(-1, 1).to(device)

qa_pairs = [
    ("nasılsın", "iyiyim teşekkür ederim sorduğun için"),
    ("senin adın ne", "benim adım yapay zeka sohbet robotu"),
    ("ne yapabilirsin", "sana basit konularda cevaplar verebilirim"),
    ("hava nasıl bugün", "üzgünüm ama hava durumunu henüz bilemiyorum"),
    ("teşekkür ederim", "rica ederim başka bir sorun var mıydı"),
    ("kaç yaşındasın", "ben bir yazılımım benim yaşım olmaz")
]
input_lang = Lang(); output_lang = Lang()
for q, a in qa_pairs: input_lang.addSentence(q); output_lang.addSentence(a)

# --- Modeller (Hata Düzeltildi ve Attention Eklendi) ---
hidden_size = 128
max_length = 20 # Maksimum cümle uzunluğu

# Encoder, cümlenin sıralı anlamını tek bir vektöre kodlar.
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        # HATA DÜZELTİLDİ: self.hidden_size burada tanımlanmalı.
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.gru(embedded, hidden)
        return output, hidden

    def initHidden(self): return torch.zeros(1, 1, self.hidden_size, device=device)

# AttnDecoderRNN, her adımda Encoder'ın çıktılarına "dikkat ederek" yeni kelime üretir.
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention = nn.Linear(hidden_size * 2, max_length)
        self.attn_combine = nn.Linear(hidden_size * 2, hidden_size)
        self.dropout = nn.Dropout(dropout_p)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)
        
        attn_weights = F.softmax(self.attention(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))
        
        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)
        output = F.relu(output)
        
        output, hidden = self.gru(output, hidden)
        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

# --- Eğitim Fonksiyonu ---
def train(inp_t, tar_t, enc, dec, enc_opt, dec_opt, crit):
    enc_h = enc.initHidden(); enc_opt.zero_grad(); dec_opt.zero_grad(); loss=0
    enc_outs = torch.zeros(max_length, enc.hidden_size, device=device)
    for ei in range(len(inp_t)):
        enc_out, enc_h = enc(inp_t[ei], enc_h)
        enc_outs[ei] = enc_out[0, 0]
        
    dec_i=torch.tensor([[SOS_token]], device=device); dec_h=enc_h
    for di in range(len(tar_t)):
        dec_o, dec_h, _ = dec(dec_i, dec_h, enc_outs)
        loss += crit(dec_o, tar_t[di]); dec_i = tar_t[di]
        if dec_i.item() == EOS_token: break
    loss.backward(); enc_opt.step(); dec_opt.step()
    return loss.item() / len(tar_t)

# --- Eğitim ---
encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)
encoder_optimizer = optim.SGD(encoder.parameters(), lr=0.01)
decoder_optimizer = optim.SGD(decoder.parameters(), lr=0.01)
criterion = nn.NLLLoss()

print("Chatbot modeli eğitiliyor...")
for i in range(10000): # Eğitim iterasyonları
    pair = random.choice(qa_pairs)
    input_tensor = sentence_to_tensor(input_lang, pair[0])
    target_tensor = sentence_to_tensor(output_lang, pair[1])
    train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
print("Eğitim tamamlandı.")


# --- UÇTAN UCA GÖSTERİM FONKSİYONU ---
def respond_and_explain(sentence):
    print(f"\n========================================================")
    print(f">>> KULLANICI SORUSU: '{sentence}'")
    print("========================================================")
    with torch.no_grad():
        # --- AŞAMA 1: EMBEDDING ---
        print("\n--- AŞAMA 1: EMBEDDING (Anlamı Sayısallaştırma) ---")
        input_tensor = sentence_to_tensor(input_lang, sentence)
        embedded_vectors = encoder.embedding(input_tensor).squeeze(1)
        print(f"'{sentence}' cümlesi, her kelime için {hidden_size} boyutlu vektörler içeren bir tensöre dönüştürüldü.")
        print(">>> KAZANIM: Metin, artık makinenin işleyebileceği bir formata sahip.\n")
        
        # Encoder
        encoder_hidden = encoder.initHidden()
        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
        for ei in range(len(input_tensor)):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] = encoder_output[0, 0]
            
        # Decoder
        decoder_input = torch.tensor([[SOS_token]], device=device)
        decoder_hidden = encoder_hidden
        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        print("\n--- AŞAMA 2 & 3: ATTENTION (Odaklanma) ve DECODER (Üretme) ---")
        for di in range(max_length):
            # Her adımda Attention ve Decoder birlikte çalışır
            decoder_output, decoder_hidden, attn_weights = decoder(decoder_input, decoder_hidden, encoder_outputs)
            
            # --- ARA ÇIKTILAR ---
            decoder_attentions[di] = attn_weights.data
            topv, topi = decoder_output.data.topk(1)
            word_index = topi.item()
            
            print(f" Adım {di+1}:")
            # AŞAMA 2'NİN KAZANIMI
            print("   - ATTENTION: Girdi cümlesindeki kelimelere dikkat odakları:")
            attention_slice = attn_weights.squeeze(0).squeeze(0).numpy()
            for i, word in enumerate(sentence.split(' ')):
                print(f"     '{word}': {attention_slice[i]:.2f}", end=" | ")
            print("\n   >>> KAZANIM: Decoder, şu an üreteceği kelime için sorunun en ilgili kısmına odaklandı.")
            
            # AŞAMA 3'ÜN KAZANIMI
            word = output_lang.index2word[word_index]
            print(f"   - DECODER: Odaklanmış bilgiyi kullanarak '{word}' kelimesini üretti.")
            print(f"   >>> KAZANIM: Cevap, kelime kelime, bağlama uygun şekilde sıfırdan inşa ediliyor.\n")
            
            if word_index == EOS_token: break
            decoded_words.append(word)
            decoder_input = topi.squeeze().detach()

        return ' '.join(decoded_words), decoder_attentions[:di + 1, :len(sentence.split(' '))]

# --- İnteraktif Sohbet ---
print("\n--- SOHBET BAŞLATILDI (Çıkmak için 'exit' yazın) ---")
user_input = "merhaba"
response, attentions = respond_and_explain(user_input)
print("\n--------------------------------------------------------")
print(f"Bot Cevabı: {response}")
print("--------------------------------------------------------")
'''
while True:
    try:
        user_input = input("Siz: ")
        if user_input.lower() == 'exit':
            break
        response, attentions = respond_and_explain(user_input)
        print("\n--------------------------------------------------------")
        print(f"Bot Cevabı: {response}")
        print("--------------------------------------------------------")
    except KeyError:
        print("Bot: Üzgünüm, bu kelimelerden bazılarını bilmiyorum. Başka bir şey sorabilir misin?")
    except Exception as e:
        print(f"Bir hata oluştu: {e}")
'''

Chatbot modeli eğitiliyor...
Eğitim tamamlandı.

--- SOHBET BAŞLATILDI (Çıkmak için 'exit' yazın) ---

>>> KULLANICI SORUSU: 'merhaba'

--- AŞAMA 1: EMBEDDING (Anlamı Sayısallaştırma) ---
'merhaba' cümlesi, her kelime için 128 boyutlu vektörler içeren bir tensöre dönüştürüldü.
>>> KAZANIM: Metin, artık makinenin işleyebileceği bir formata sahip.


--- AŞAMA 2 & 3: ATTENTION (Odaklanma) ve DECODER (Üretme) ---
 Adım 1:
   - ATTENTION: Girdi cümlesindeki kelimelere dikkat odakları:
     'merhaba': 0.34 | 
   >>> KAZANIM: Decoder, şu an üreteceği kelime için sorunun en ilgili kısmına odaklandı.
   - DECODER: Odaklanmış bilgiyi kullanarak 'sana' kelimesini üretti.
   >>> KAZANIM: Cevap, kelime kelime, bağlama uygun şekilde sıfırdan inşa ediliyor.

 Adım 2:
   - ATTENTION: Girdi cümlesindeki kelimelere dikkat odakları:
     'merhaba': 0.06 | 
   >>> KAZANIM: Decoder, şu an üreteceği kelime için sorunun en ilgili kısmına odaklandı.
   - DECODER: Odaklanmış bilgiyi kullanarak 'basit' kelimesi

'\nwhile True:\n    try:\n        user_input = input("Siz: ")\n        if user_input.lower() == \'exit\':\n            break\n        response, attentions = respond_and_explain(user_input)\n        print("\n--------------------------------------------------------")\n        print(f"Bot Cevabı: {response}")\n        print("--------------------------------------------------------")\n    except KeyError:\n        print("Bot: Üzgünüm, bu kelimelerden bazılarını bilmiyorum. Başka bir şey sorabilir misin?")\n    except Exception as e:\n        print(f"Bir hata oluştu: {e}")\n'

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import torch.nn.functional as F
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# --- Genel Kurulum ---
device = torch.device("cpu")
SOS_token = 0; EOS_token = 1; UNK_token = 2

# Daha uzun ve çeşitli cümleler içeren veri seti
qa_pairs = [
    ("nasılsın", "iyiyim teşekkür ederim sorduğun için"),
    ("senin adın ne", "benim adım yapay zeka sohbet robotu"),
    ("ne yapabilirsin", "sana basit konularda doğru cevaplar verebilirim"),
    ("bugün hava nasıl", "üzgünüm ama hava durumunu henüz bilemiyorum"),
    ("en sevdiğin renk ne", "bir yazılım olarak renkleri göremem"),
    ("bana bir şaka anlat", "neden yapay zeka hiç kaybolmaz çünkü her zaman evdeki sunucusuna döner"),
    ("teşekkürler", "rica ederim başka bir sorun var mıydı")
]

class Lang:
    def __init__(self): self.word2index = {}; self.index2word = {0:"SOS", 1:"EOS", 2:"<UNK>"}; self.n_words = 3
    def addSentence(self, sentence):
        for word in sentence.split(' '): self.addWord(word)
    def addWord(self, word):
        if word not in self.word2index: self.word2index[word]=self.n_words; self.index2word[self.n_words]=word; self.n_words+=1

input_lang = Lang(); output_lang = Lang()
for q, a in qa_pairs: input_lang.addSentence(q); output_lang.addSentence(a)

def sentence_to_tensor(lang, sentence):
    indexes = [lang.word2index.get(word, UNK_token) for word in sentence.split(' ')]
    indexes.append(EOS_token)
    return torch.LongTensor(indexes).view(-1, 1).to(device)

# --- AŞAMA 1: Sadece Embedding Kullanan "Kelime Avcısı" Bot ---
print("="*80)
print("--- Bot 1: 'Kelime Avcısı' (Sadece Embedding Yeteneği) ---")

def simple_keyword_bot(user_input, qa_database):
    print(f"\n[Bot 1] Kullanıcı Sorusu: '{user_input}'")
    input_words = set(user_input.split(' '))
    best_match_score = 0
    best_answer = "Üzgünüm, bu konuda bir bilgim yok."

    for question, answer in qa_database:
        question_words = set(question.split(' '))
        match_score = len(input_words.intersection(question_words))
        if match_score > best_match_score:
            best_match_score = match_score
            best_answer = answer
            
    print(f"[Bot 1] Cevap (Hazır Cevaplardan Seçim): '{best_answer}'")
    print("\n>>> KAZANIM: Embedding'in temel fikri olan kelime eşleştirme sayesinde basit bir cevap bulabildik.")
    print(">>> EKSİKLİK: Cümle yapısı, gramer veya bağlam tamamen yok sayıldı. Yeni bir cümle kuramıyor, sadece ezberindekini tekrar ediyor.")

# --- AŞAMA 2 & 3 için Ortak Modeller ---
hidden_size = 128
max_length = 25

class Encoder(nn.Module):
    def __init__(self, i_s, h_s): super(Encoder, self).__init__(); self.h_s=h_s; self.emb=nn.Embedding(i_s,h_s); self.gru=nn.GRU(h_s,h_s)
    def forward(self, i, h): e=self.emb(i).view(1,1,-1); o,h=self.gru(e,h); return o,h
    def initHidden(self): return torch.zeros(1, 1, self.h_s, device=device)

class Decoder(nn.Module): # Attention'SIZ Decoder
    def __init__(self, h_s, o_s): super(Decoder, self).__init__(); self.emb=nn.Embedding(o_s,h_s); self.gru=nn.GRU(h_s,h_s); self.out=nn.Linear(h_s,o_s); self.softmax=nn.LogSoftmax(dim=1)
    def forward(self, i, h): o=self.emb(i).view(1,1,-1); o=F.relu(o); o,h=self.gru(o,h); o=self.softmax(self.out(o[0])); return o,h
    
class AttnDecoder(nn.Module): # Attention'LI Decoder
    def __init__(self, h_s, o_s, dropout_p=0.1):
        super(AttnDecoder, self).__init__(); self.emb=nn.Embedding(o_s,h_s); self.attn=nn.Linear(h_s*2,max_length); self.attn_combine=nn.Linear(h_s*2,h_s); self.dropout=nn.Dropout(dropout_p); self.gru=nn.GRU(h_s,h_s); self.out=nn.Linear(h_s,o_s)
    def forward(self, i, h, enc_outs):
        e=self.emb(i).view(1,1,-1); e=self.dropout(e)
        aw=F.softmax(self.attn(torch.cat((e[0],h[0]),1)),dim=1); aa=torch.bmm(aw.unsqueeze(0),enc_outs.unsqueeze(0))
        o=torch.cat((e[0],aa[0]),1); o=self.attn_combine(o).unsqueeze(0); o=F.relu(o)
        o,h=self.gru(o,h); o=F.log_softmax(self.out(o[0]),dim=1); return o,h,aw

# Eğitim fonksiyonu
def train_seq2seq(inp_t, tar_t, enc, dec, enc_opt, dec_opt, crit, use_attention=False):
    enc_h=enc.initHidden(); enc_opt.zero_grad(); dec_opt.zero_grad(); loss=0
    enc_outs=torch.zeros(max_length,enc.h_s,device=device)
    for ei in range(len(inp_t)): enc_o,enc_h = enc(inp_t[ei],enc_h); enc_outs[ei]=enc_o[0,0]
    dec_i=torch.tensor([[SOS_token]],device=device); dec_h=enc_h
    for di in range(len(tar_t)):
        if use_attention: dec_o, dec_h, _ = dec(dec_i, dec_h, enc_outs)
        else: dec_o, dec_h = dec(dec_i, dec_h)
        loss+=crit(dec_o,tar_t[di]); dec_i=tar_t[di]
        if dec_i.item() == EOS_token: break
    loss.backward(); enc_opt.step(); dec_opt.step()

# Değerlendirme fonksiyonu
def evaluate(sentence, enc, dec, use_attention=False, temperature=0.5):
    with torch.no_grad():
        inp_t=sentence_to_tensor(input_lang,sentence); enc_h=enc.initHidden()
        enc_outs=torch.zeros(max_length,enc.h_s,device=device)
        for ei in range(len(inp_t)): enc_o,enc_h=enc(inp_t[ei],enc_h); enc_outs[ei]=enc_o[0,0]
        dec_i=torch.tensor([[SOS_token]],device=device); dec_h=enc_h
        decoded_words=[]
        for di in range(max_length):
            if use_attention: dec_o,dec_h,_ = dec(dec_i,dec_h,enc_outs)
            else: dec_o,dec_h = dec(dec_i,dec_h)
            
            # YARATICILIK İÇİN TEMPERATURE SAMPLING
            decoder_output_dist = dec_o.div(temperature).exp()
            topi = torch.multinomial(decoder_output_dist, 1)[0]
            
            if topi.item() == EOS_token: break
            else: decoded_words.append(output_lang.index2word[topi.item()])
            dec_i = topi.squeeze().detach()
        return ' '.join(decoded_words)

# --- Bot 2 ve 3 için Eğitim ---
print("\n" + "="*80)
print("--- Bot 2 ve Bot 3 için Modeller Eğitiliyor ---")
# Bot 2 Modelleri (Attention'sız)
encoder2 = Encoder(input_lang.n_words, hidden_size).to(device)
decoder2 = Decoder(hidden_size, output_lang.n_words).to(device)
enc2_opt = optim.SGD(encoder2.parameters(), lr=0.01); dec2_opt = optim.SGD(decoder2.parameters(), lr=0.01)

# Bot 3 Modelleri (Attention'lı)
encoder3 = Encoder(input_lang.n_words, hidden_size).to(device)
decoder3 = AttnDecoder(hidden_size, output_lang.n_words).to(device)
enc3_opt = optim.SGD(encoder3.parameters(), lr=0.01); dec3_opt = optim.SGD(decoder3.parameters(), lr=0.01)

criterion = nn.NLLLoss()
for i in range(10000):
    pair = random.choice(qa_pairs)
    inp_t = sentence_to_tensor(input_lang, pair[0]); tar_t = sentence_to_tensor(output_lang, pair[1])
    train_seq2seq(inp_t,tar_t,encoder2,decoder2,enc2_opt,dec2_opt,criterion,use_attention=False)
    train_seq2seq(inp_t,tar_t,encoder3,decoder3,enc3_opt,dec3_opt,criterion,use_attention=True)
print("Eğitimler tamamlandı.")

# --- KARŞILAŞTIRMALI TEST ---
print("\n" + "="*80)
print("--- MODELLERİN KARŞILAŞTIRILMASI ---")

# Uzun ve detaylı bir soru seçelim
test_sentence = "en sevdiğin renk ne olabilir"

# --- Bot 1'in Cevabı ---
simple_keyword_bot(test_sentence, qa_pairs)

# --- Bot 2'nin Cevabı ---
print(f"\n[Bot 2] Kullanıcı Sorusu: '{test_sentence}'")
response2 = evaluate(test_sentence, encoder2, decoder2, use_attention=False)
print(f"[Bot 2] Cevap (Attention'sız Encoder-Decoder): '{response2}'")
print("\n>>> KAZANIM: Artık ezberlemiyor, kelime kelime yeni bir cümle üretiyoruz.")
print(">>> EKSİKLİK: Cümle uzun olduğunda, Encoder'ın ürettiği tek bir 'özet vektörü' yetersiz kalıyor. Model, cümlenin başındaki 'renk' gibi önemli bir detayı unutabiliyor ve genel bir cevap veriyor.")

# --- Bot 3'ün Cevabı ---
print(f"\n[Bot 3] Kullanıcı Sorusu: '{test_sentence}'")
print("Not: Yaratıcılık için Temperature Sampling kullanıldığından, her seferinde biraz farklı cevaplar alabilirsiniz.")
response3 = evaluate(test_sentence, encoder3, decoder3, use_attention=True, temperature=0.7)
print(f"[Bot 3] Cevap (Attention'lı Encoder-Decoder): '{response3}'")
print("\n>>> KAZANIM: Attention sayesinde Decoder, cevabı üretirken sorunun en kilit kısmına ('renk' kelimesine) odaklanabildi ve bu sayede çok daha isabetli bir cevap üretti.")
print(">>> KAZANIM: Sıcaklık (temperature) parametresi, modele deterministik olmak yerine olasılıksal ve yaratıcı cevaplar üretme yeteneği kazandırdı.")

print("\n" + "="*80)
print("SONUÇ: Embedding ile başladık, Encoder-Decoder ile cümle kurmayı öğrendik ve son olarak Attention ile bu yeteneği akıllı ve odaklı hale getirdik.")

--- Bot 1: 'Kelime Avcısı' (Sadece Embedding Yeteneği) ---

--- Bot 2 ve Bot 3 için Modeller Eğitiliyor ---
Eğitimler tamamlandı.

--- MODELLERİN KARŞILAŞTIRILMASI ---

[Bot 1] Kullanıcı Sorusu: 'en sevdiğin renk ne olabilir'
[Bot 1] Cevap (Hazır Cevaplardan Seçim): 'bir yazılım olarak renkleri göremem'

>>> KAZANIM: Embedding'in temel fikri olan kelime eşleştirme sayesinde basit bir cevap bulabildik.
>>> EKSİKLİK: Cümle yapısı, gramer veya bağlam tamamen yok sayıldı. Yeni bir cümle kuramıyor, sadece ezberindekini tekrar ediyor.

[Bot 2] Kullanıcı Sorusu: 'en sevdiğin renk ne olabilir'
[Bot 2] Cevap (Attention'sız Encoder-Decoder): 'bir yazılım olarak renkleri göremem'

>>> KAZANIM: Artık ezberlemiyor, kelime kelime yeni bir cümle üretiyoruz.
>>> EKSİKLİK: Cümle uzun olduğunda, Encoder'ın ürettiği tek bir 'özet vektörü' yetersiz kalıyor. Model, cümlenin başındaki 'renk' gibi önemli bir detayı unutabiliyor ve genel bir cevap veriyor.

[Bot 3] Kullanıcı Sorusu: 'en sevdiğin renk ne olabi